In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_imdb_movies(max_pages=1):
    """
    Scrapea la lista de películas en IMDb (por popularidad ascendente),
    iterando sobre 'max_pages' páginas.
    """
    # URL base sin el parámetro de página
    base_url = "https://www.imdb.com/list/ls009072108/"
    
    # Cabeceras para simular un navegador
    headers = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
            " AppleWebKit/537.36 (KHTML, like Gecko)"
            " Chrome/112.0.0.0 Safari/537.36"
        )
    }
    
    all_movies_data = []
    page = 1
    
    while page <= max_pages:
        # Construimos la URL con el número de página
        # Ejemplo: ?sort=popularity,asc&page=2
        url = f"{base_url}?sort=popularity,asc&page={page}"
        print(f"Scrapeando página {page}: {url}")
        
        response = requests.get(url, headers=headers)
        # Si da status != 200, salimos
        if response.status_code != 200:
            print(f"Status {response.status_code}. Se detiene la extracción.")
            break
        
        soup = BeautifulSoup(response.text, 'html.parser')
        movie_items = soup.select('li.ipc-metadata-list-summary-item')
        
        # Si no hay películas en la página, dejamos de recorrer
        if not movie_items:
            print("No se encontraron más películas. Terminando.")
            break
        
        # Recorremos las películas de la página
        for movie in movie_items:
            try:
                # Título
                title_tag = movie.select_one('h3.ipc-title__text')
                title = title_tag.get_text(strip=True) if title_tag else "No Title"
                
                # Año, duración
                metadata_spans = movie.select('div.sc-d5ea4b9d-6 span.sc-d5ea4b9d-7')
                year = metadata_spans[0].get_text(strip=True) if len(metadata_spans) > 0 else ""
                duration = metadata_spans[1].get_text(strip=True) if len(metadata_spans) > 1 else ""
                
                # Rating IMDb
                rating_span = movie.select_one('span.ipc-rating-star--rating')
                rating = rating_span.get_text(strip=True) if rating_span else ""
                
                # Poster
                poster_img_tag = movie.select_one('div.ipc-poster__poster-image img')
                poster_src = poster_img_tag.get('src') if poster_img_tag else ""
                
                # Presupuesto
                budget_text = ""
                budget_div = movie.select_one('div[data-testid="title-list-item-description"]')
                if budget_div:
                    text_all = budget_div.get_text(strip=True)
                    # e.g. "Budget: $290,000,000"
                    if "Budget:" in text_all:
                        budget_text = text_all.replace("Budget: ", "")
                
                all_movies_data.append({
                    'Title': title,
                    'Year': year,
                    'Duration': duration,
                    'Rating': rating,
                    'Budget': budget_text,
                    'PosterURL': poster_src
                })
            except Exception as e:
                print(f"Error procesando una película: {e}")
                continue
        
        # Incrementamos la página
        page += 1
    
    # Convertimos todo a DataFrame
    df = pd.DataFrame(all_movies_data)
    return df

if __name__ == "__main__":
    # Ejemplo: si queremos scrapear hasta 3 páginas
    df_imdb = scrape_imdb_movies(max_pages=3)
    print(df_imdb.head())
    
    # Guardamos en CSV
    df_imdb.to_csv('../data/imdb_movies_budgets.csv', index=False)
    print(f"Guardado CSV con {len(df_imdb)} registros.")


Scrapeando página 1: https://www.imdb.com/list/ls009072108/?sort=popularity,asc&page=1
Scrapeando página 2: https://www.imdb.com/list/ls009072108/?sort=popularity,asc&page=2
Status 404. Se detiene la extracción.
                                        Title  Year Duration Rating  \
0                                  2. Titanic  1997   3h 14m    7.9   
1                     35. El caballero oscuro  2008   2h 32m    9.0   
2                                   23. Troya  2004   2h 43m    7.3   
3  17. El caballero oscuro: La leyenda renace  2012   2h 44m    8.4   
4                          18. Los Vengadores  2012   2h 23m    8.0   

         Budget                                          PosterURL  
0  $290,000,000  https://m.media-amazon.com/images/M/MV5BNjQ4NG...  
1  $200,000,000  https://m.media-amazon.com/images/M/MV5BMTMxNT...  
2  $215,000,000  https://m.media-amazon.com/images/M/MV5BMTk5Mz...  
3  $230,000,000  https://m.media-amazon.com/images/M/MV5BMTk4OD...  
4  $220,000,000 